In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta, date
from tqdm import tqdm_notebook as tqdm
import seaborn as sns
import pickle

from multiprocessing import Pool
from functools import partial
import Training_function as func

In [2]:
data = pd.read_csv('D:\\庫存健診開發\\data\\processed\\TWSE_Stock_feature.csv', converters={'ts': str, 'StockNo': str, 'StockName': str})
data['ts'] = pd.to_datetime(data['ts'])
data

,ts,StockNo,StockName,open,high,low,close,vol,total,capital,...,investment_buy_kurtosis,dealer_buy_skew,dealer_buy_kurtosis,pvt_current,pvt,TR,ATR,RSI_15,SO5,SO3
0,2007-07-02,1103,嘉泥,16.690909,17.075348,16.389434,16.696833,8922.148890,241197.752334,163.0,...,NaN,NaN,NaN,NaN,NaN,0.685913,0.685913,0.000000,0.000000,0.000000
1,2007-07-03,1103,嘉泥,16.792673,17.181600,16.483122,16.793699,9528.445464,252811.658292,161.7,...,NaN,NaN,NaN,55.278694,NaN,0.698478,0.692614,0.000000,0.000000,0.000000
2,2007-07-04,1103,嘉泥,16.895752,17.289264,16.578034,16.891876,11023.590605,286518.072056,160.7,...,NaN,NaN,NaN,64.444429,119.723123,0.711230,0.699664,0.000000,0.000000,0.000000
3,2007-07-05,1103,嘉泥,16.999760,17.397944,16.673817,16.991004,12925.887006,327957.933786,163.7,...,NaN,NaN,NaN,75.854092,140.298521,0.724127,0.707054,0.000000,0.000000,0.000000
4,2007-07-06,1103,嘉泥,17.109628,17.512718,16.774992,17.095685,22855.487064,576534.951942,175.2,...,NaN,NaN,NaN,140.811720,216.665812,0.737727,0.714925,0.000000,0.628737,0.324509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2369242,2019-09-17,9955,佳龍,19.540854,19.684652,19.311951,19.442533,809.447597,15833.513865,19.9,...,-3.0,-4.129483,15.052632,-6.151946,-13.724452,0.372701,0.503662,0.060364,0.119488,0.113594
2369243,2019-09-18,9955,佳龍,19.327445,19.472854,19.141697,19.287597,362.143798,6863.474058,19.8,...,-3.0,-4.129483,15.052632,-2.885898,-9.037843,0.331157,0.482099,0.049477,0.134196,0.123895
2369244,2019-09-19,9955,佳龍,19.170038,19.316798,19.013120,19.169869,840.337000,15650.591246,19.4,...,-3.0,-4.129483,15.052632,-5.129252,-8.015149,0.303678,0.459797,0.042812,0.150675,0.137285
2369245,2019-09-20,9955,佳龍,19.072044,19.220740,18.930397,19.093576,434.980049,8042.232935,19.8,...,-3.0,-4.129483,15.052632,-1.731143,-6.860395,0.290343,0.438615,0.038954,0.171759,0.154522


In [3]:
train_df = pd.read_csv('D:\\庫存健診開發\\data\\Clustering\\train_cluster.csv', index_col=0)
test_df = pd.read_csv('D:\\庫存健診開發\\data\\Clustering\\test_cluster.csv', index_col=0)

In [5]:
file_path = 'D:\\庫存健診開發\\data\\Training\\Raw\\'
func.total_split(train_df, test_df, data, file_path)

D:\庫存健診開發\code\TrainingPrep\Training_function.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['cluster'] = i
D:\庫存健診開發\code\TrainingPrep\Training_function.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['cluster'] = i
